Install and import library modules. Download English stopwords and WordNet. Set logging to INFO

In [1]:
import sys
!{sys.executable} -m pip install gensim
!{sys.executable} -m pip install pyLDAvis
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install matplotlib
!{sys.executable} -m pip install nltk

import re, gensim, pyLDAvis, logging, datetime, multiprocessing, os, pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from math import floor, ceil
from nltk.corpus import stopwords
from nltk import download
from nltk.stem import WordNetLemmatizer

download('stopwords')
download('wordnet')

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

You should consider upgrading via the '/usr/local/Cellar/jupyterlab/2.1.2/libexec/bin/python3.8 -m pip install --upgrade pip' command.
You should consider upgrading via the '/usr/local/Cellar/jupyterlab/2.1.2/libexec/bin/python3.8 -m pip install --upgrade pip' command.
You should consider upgrading via the '/usr/local/Cellar/jupyterlab/2.1.2/libexec/bin/python3.8 -m pip install --upgrade pip' command.
You should consider upgrading via the '/usr/local/Cellar/jupyterlab/2.1.2/libexec/bin/python3.8 -m pip install --upgrade pip' command.
You should consider upgrading via the '/usr/local/Cellar/jupyterlab/2.1.2/libexec/bin/python3.8 -m pip install --upgrade pip' command.
You should consider upgrading via the '/usr/local/Cellar/jupyterlab/2.1.2/libexec/bin/python3.8 -m pip install --upgrade pip' command.


[nltk_data] Downloading package stopwords to /Users/joost/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/joost/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Read the data as pandas dataframe, remove items with missing abstracts, and fill other missing data with empty strings

In [2]:
df = pd.read_csv('data/data_merged_and_filtered.csv')
df = df[df['abstract'].notna()]
df.fillna("", inplace=True)

Combine the text fiels into one

In [3]:
df["data"] = df["abstract"] + " " + df["keywords"] + " " + df["title"]

2021-07-18 19:57:10,824 : INFO : NumExpr defaulting to 8 threads.


Remove copyright notices from the text

In [4]:
copyright_re = "(©|&#x00A9;|Copyright)(.*?(?=B\\.V\\.|Ltd|\\.))(B\\.V\\.|Ltd|\\.)"
rights = "All rights reserved."
df.replace(copyright_re, '', regex=True, inplace=True)
df.replace(rights, '', regex=False, inplace=True)

Tokenize, convert to lowercase, remove punctuation, remove stop-words, lemmatize

In [5]:
def preprocess(text):
  result = [word for word in gensim.utils.simple_preprocess(text, deacc=True) if word not in stop_words]
  wordnet_lemmatizer = WordNetLemmatizer()
  result = [wordnet_lemmatizer.lemmatize(word) for word in result]
  return result

stop_words = stopwords.words('english')
df["data_preprocessed"] = df["data"].map(preprocess)

Generate the Dictionairy and perform relative pruning

In [6]:
dictionary = gensim.corpora.Dictionary(df["data_preprocessed"])
dictionary.filter_extremes(no_below=(floor(0.005 * len(df))), no_above=0.99)

2021-07-18 19:59:49,600 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2021-07-18 19:59:51,356 : INFO : adding document #10000 to Dictionary(27074 unique tokens: ['ability', 'acceptable', 'additionally', 'analysis', 'anti']...)
2021-07-18 19:59:52,991 : INFO : adding document #20000 to Dictionary(38089 unique tokens: ['ability', 'acceptable', 'additionally', 'analysis', 'anti']...)
2021-07-18 19:59:54,538 : INFO : adding document #30000 to Dictionary(46717 unique tokens: ['ability', 'acceptable', 'additionally', 'analysis', 'anti']...)
2021-07-18 19:59:56,098 : INFO : adding document #40000 to Dictionary(53696 unique tokens: ['ability', 'acceptable', 'additionally', 'analysis', 'anti']...)
2021-07-18 19:59:57,589 : INFO : adding document #50000 to Dictionary(59648 unique tokens: ['ability', 'acceptable', 'additionally', 'analysis', 'anti']...)
2021-07-18 19:59:59,102 : INFO : adding document #60000 to Dictionary(64997 unique tokens: ['ability', 'acceptable', 'additional

Make the bag of words

In [7]:
corpus = [dictionary.doc2bow(doc) for doc in df["data_preprocessed"]]

Generate all models for evaluation

In [ ]:
k = [1, 4, 9, 16, 25, 36, 49, 64, 81, 100, 121, 144, 169, 196, 225, 256, 289, 324, 361] #later added [30, 40, 42]
random_states = [24, 1024] #later added [27, 1027]

texts = df["data_preprocessed"]
chunksize = ceil(len(texts))
passes = 20
iterations = 100
eval_every = None
dir_prior = 'auto'

for random_state in random_states:
    for num_topics in k:
        
        name = "rs{}k{}".format(random_state, num_topics)
        
        
        if 'model_' + name in os.listdir('models'):
            print(datetime.datetime.now().strftime('%H:%M:%S') + ' Model ' + name + ' already in directory. Skipped')
        
        else:
            print(datetime.datetime.now().strftime('%H:%M:%S') + " Building model " + name)
            lda_model = gensim.models.LdaModel(corpus, num_topics=num_topics, id2word=dictionary, alpha=dir_prior, eta=dir_prior, random_state=random_state, chunksize=chunksize, passes=passes, iterations=iterations)

            lda_model.save(os.path.join('models', 'model_' + name))

2021-07-18 22:42:43,370 : INFO : using autotuned alpha, starting with [0.0034602077, 0.0034602077, 0.0034602077, 0.0034602077, 0.0034602077, 0.0034602077, 0.0034602077, 0.0034602077, 0.0034602077, 0.0034602077, 0.0034602077, 0.0034602077, 0.0034602077, 0.0034602077, 0.0034602077, 0.0034602077, 0.0034602077, 0.0034602077, 0.0034602077, 0.0034602077, 0.0034602077, 0.0034602077, 0.0034602077, 0.0034602077, 0.0034602077, 0.0034602077, 0.0034602077, 0.0034602077, 0.0034602077, 0.0034602077, 0.0034602077, 0.0034602077, 0.0034602077, 0.0034602077, 0.0034602077, 0.0034602077, 0.0034602077, 0.0034602077, 0.0034602077, 0.0034602077, 0.0034602077, 0.0034602077, 0.0034602077, 0.0034602077, 0.0034602077, 0.0034602077, 0.0034602077, 0.0034602077, 0.0034602077, 0.0034602077, 0.0034602077, 0.0034602077, 0.0034602077, 0.0034602077, 0.0034602077, 0.0034602077, 0.0034602077, 0.0034602077, 0.0034602077, 0.0034602077, 0.0034602077, 0.0034602077, 0.0034602077, 0.0034602077, 0.0034602077, 0.0034602077, 0.003

22:42:43 Model rs24k1 already in directory. Skipped
22:42:43 Model rs24k4 already in directory. Skipped
22:42:43 Model rs24k9 already in directory. Skipped
22:42:43 Model rs24k16 already in directory. Skipped
22:42:43 Model rs24k25 already in directory. Skipped
22:42:43 Model rs24k36 already in directory. Skipped
22:42:43 Model rs24k49 already in directory. Skipped
22:42:43 Model rs24k64 already in directory. Skipped
22:42:43 Model rs24k81 already in directory. Skipped
22:42:43 Model rs24k100 already in directory. Skipped
22:42:43 Model rs24k121 already in directory. Skipped
22:42:43 Model rs24k144 already in directory. Skipped
22:42:43 Model rs24k169 already in directory. Skipped
22:42:43 Model rs24k196 already in directory. Skipped
22:42:43 Model rs24k225 already in directory. Skipped
22:42:43 Model rs24k256 already in directory. Skipped
22:42:43 Building model rs24k289


2021-07-18 22:51:09,146 : INFO : -8.923 per-word bound, 485.4 perplexity estimate based on a held-out corpus of 79433 documents with 7925243 words
2021-07-18 22:51:09,147 : INFO : PROGRESS: pass 0, at document #79433/79433
2021-07-18 23:01:06,044 : INFO : optimized alpha [0.0034119778, 0.0034170395, 0.0034213378, 0.00342126, 0.0034124872, 0.0034547416, 0.0034179427, 0.003429196, 0.0034399536, 0.0034118043, 0.003413809, 0.0034044646, 0.0034074676, 0.0034098143, 0.0034275514, 0.00341662, 0.0034199606, 0.003415651, 0.0034064562, 0.0034163145, 0.003469709, 0.0034065293, 0.003423617, 0.003422687, 0.0034181576, 0.0034282508, 0.0034115696, 0.0034342317, 0.003414658, 0.003411818, 0.0034523355, 0.0034197296, 0.0034078646, 0.0034270096, 0.0034314278, 0.0034357992, 0.003425862, 0.003417137, 0.003412755, 0.003424819, 0.0034250624, 0.0034085202, 0.0034189723, 0.0034080627, 0.003412198, 0.003411207, 0.0034369293, 0.0034177173, 0.0034190728, 0.0034293546, 0.0034121163, 0.0034168486, 0.0034182598, 0.0

2021-07-18 23:16:08,618 : INFO : topic #109 (0.003): 0.026*"privacy" + 0.019*"information" + 0.012*"access" + 0.012*"control" + 0.011*"data" + 0.009*"security" + 0.008*"authentication" + 0.008*"based" + 0.007*"scheme" + 0.007*"protocol"
2021-07-18 23:16:08,620 : INFO : topic #166 (0.003): 0.041*"attack" + 0.018*"channel" + 0.015*"side" + 0.014*"fault" + 0.014*"aes" + 0.013*"differential" + 0.012*"analysis" + 0.012*"privacy" + 0.011*"security" + 0.009*"model"
2021-07-18 23:16:08,621 : INFO : topic #229 (0.004): 0.105*"data" + 0.038*"privacy" + 0.015*"preserving" + 0.009*"method" + 0.009*"model" + 0.008*"mining" + 0.008*"algorithm" + 0.008*"based" + 0.007*"information" + 0.007*"approach"
2021-07-18 23:16:08,622 : INFO : topic #241 (0.003): 0.106*"data" + 0.038*"privacy" + 0.026*"big" + 0.015*"mining" + 0.011*"information" + 0.008*"preserving" + 0.008*"protection" + 0.008*"paper" + 0.007*"technique" + 0.007*"security"
2021-07-18 23:16:08,623 : INFO : topic #77 (0.003): 0.064*"privacy" + 0

2021-07-18 23:40:59,043 : INFO : topic #11 (0.003): 0.050*"news" + 0.039*"transmission" + 0.033*"coverage" + 0.030*"data" + 0.019*"product" + 0.014*"elderly" + 0.012*"network" + 0.011*"privacy" + 0.010*"library" + 0.010*"study"
2021-07-18 23:40:59,045 : INFO : topic #25 (0.003): 0.039*"privacy" + 0.011*"data" + 0.011*"user" + 0.010*"security" + 0.009*"protocol" + 0.009*"scheme" + 0.009*"counter" + 0.007*"information" + 0.007*"based" + 0.007*"research"
2021-07-18 23:40:59,046 : INFO : topic #18 (0.004): 0.065*"online" + 0.052*"consumer" + 0.022*"commerce" + 0.021*"privacy" + 0.020*"internet" + 0.019*"study" + 0.018*"intention" + 0.018*"perceived" + 0.015*"factor" + 0.014*"customer"
2021-07-18 23:40:59,048 : INFO : topic #39 (0.004): 0.086*"protocol" + 0.036*"secure" + 0.028*"party" + 0.026*"computation" + 0.021*"privacy" + 0.015*"security" + 0.012*"based" + 0.012*"preserving" + 0.012*"two" + 0.011*"multi"
2021-07-18 23:40:59,049 : INFO : topic #229 (0.004): 0.128*"data" + 0.045*"privacy

2021-07-18 23:57:07,775 : INFO : topic #97 (0.003): 0.025*"privacy" + 0.022*"express" + 0.012*"security" + 0.012*"system" + 0.011*"application" + 0.009*"information" + 0.008*"website" + 0.008*"user" + 0.008*"private" + 0.008*"time"
2021-07-18 23:57:07,776 : INFO : topic #235 (0.003): 0.129*"driving" + 0.026*"privacy" + 0.020*"system" + 0.015*"driver" + 0.014*"computing" + 0.010*"data" + 0.007*"architecture" + 0.007*"user" + 0.007*"security" + 0.007*"paper"
2021-07-18 23:57:07,778 : INFO : topic #18 (0.004): 0.071*"online" + 0.063*"consumer" + 0.027*"commerce" + 0.021*"intention" + 0.020*"privacy" + 0.020*"perceived" + 0.019*"study" + 0.019*"internet" + 0.016*"customer" + 0.014*"factor"
2021-07-18 23:57:07,779 : INFO : topic #179 (0.004): 0.171*"cloud" + 0.097*"computing" + 0.043*"service" + 0.037*"security" + 0.018*"privacy" + 0.015*"data" + 0.012*"provider" + 0.011*"issue" + 0.011*"environment" + 0.010*"user"
2021-07-18 23:57:07,781 : INFO : topic #39 (0.004): 0.097*"protocol" + 0.043

2021-07-19 00:09:56,244 : INFO : topic #215 (0.003): 0.033*"security" + 0.022*"information" + 0.022*"privacy" + 0.017*"data" + 0.011*"system" + 0.010*"protection" + 0.009*"user" + 0.009*"based" + 0.009*"remote" + 0.008*"protocol"
2021-07-19 00:09:56,245 : INFO : topic #100 (0.003): 0.035*"data" + 0.017*"blockchain" + 0.014*"security" + 0.014*"privacy" + 0.012*"application" + 0.012*"based" + 0.012*"content" + 0.012*"patient" + 0.010*"information" + 0.009*"system"
2021-07-19 00:09:56,246 : INFO : topic #179 (0.004): 0.187*"cloud" + 0.112*"computing" + 0.041*"service" + 0.036*"security" + 0.018*"privacy" + 0.014*"data" + 0.012*"environment" + 0.012*"issue" + 0.012*"provider" + 0.009*"user"
2021-07-19 00:09:56,247 : INFO : topic #18 (0.004): 0.081*"online" + 0.074*"consumer" + 0.032*"commerce" + 0.023*"intention" + 0.022*"perceived" + 0.020*"privacy" + 0.019*"study" + 0.017*"internet" + 0.016*"website" + 0.014*"customer"
2021-07-19 00:09:56,248 : INFO : topic #39 (0.004): 0.104*"protocol" 

Evaluate all models present in the current directory

In [ ]:
models_to_evaluate = []

for entry in os.scandir('models'):
    if entry.path.endswith('.state'):
        models_to_evaluate.append(entry.path[:-6])
        
for path in models_to_evaluate:
    model = gensim.models.LdaModel.load(path)
    coherence_model_lda = gensim.models.coherencemodel.CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
    coherence = coherence_model_lda.get_coherence()
        
    with open(os.path.join('models', 'scores.txt'), "a") as f:
        f.write("\"" + str(os.path.split(path)[1]) + "\"" ' : ' + str(coherence) + ',\n')   

Save the preprocessed data, corpus and dictionary for later exploration  

In [ ]:
df.to_csv('data/data_for_interpretation.csv')

with open('models/corpus.pkl', 'wb') as f:
    pickle.dump(corpus, f)
    
dictionary.save('models/dictionary')